This notebook refered to Dandelion's "EDA+Regression" for self study.
The comment is writen by korean.

필요한 패키지를 임포트하고, csv 데이터를 읽어 옴

In [ ]:

import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as pl
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv('../input/insurance.csv')


head를 통하여 데이터를 확인함

In [ ]:
data.head()

실제로 모델 결과에 큰 영향을 주는 Null값을 칼람 별로 확인

In [ ]:
data.isnull().sum()

다행이도 Null 값이 없음.

데이터 탐색을 본격적으로 시작.

먼저 카테고리 데이터를 encoding을 통하여 수자로 변환함.

In [ ]:
from sklearn.preprocessing import LabelEncoder
#sex
le = LabelEncoder()
le.fit(data.sex.drop_duplicates()) 
data.sex = le.transform(data.sex)
# smoker or not
le.fit(data.smoker.drop_duplicates()) 
data.smoker = le.transform(data.smoker)
#region
le.fit(data.region.drop_duplicates()) 
data.region = le.transform(data.region)

Region같은 경우 변수가 적음으로 label encoding을 사용했지만, 만약에 변수가 많을 경우에는 onehot encoding을 추천함.

charge와 기타 칼람간의 상간 관계를 분석함.

In [ ]:
data.corr()['charges'].sort_values()

상관 관계는 아래와 같이 heat map을 통하여 좀 더 직관적으로 확인 가능함.

In [ ]:

f, ax = pl.subplots(figsize=(10, 8))
corr = data.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(240,10,as_cmap=True),
            square=True, ax=ax)

확인 결과 흡연이 charge랑 가장 큰 관계가 있다를 확인함.

charge의 분포를 분석하고 평균치와 비교하기 위하여 필요한 패키지를 임포트하고 분석 시작함.

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
import scipy.special
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
p = figure(title="Distribution of charges",tools="save",
            background_fill_color="#E8DDCB")
hist, edges = np.histogram(data.charges)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
p.xaxis.axis_label = 'x'
p.yaxis.axis_label = 'Pr(x)'
show(p)

흡연이 charge에 큰 영향을 준다를 앞에서 결론을 얻었고, 흡연과 비 흡연자 들의 charge 분포를 확인 함

In [ ]:
f= pl.figure(figsize=(12,5))

ax=f.add_subplot(121)
sns.distplot(data[(data.smoker == 1)]["charges"],color='c',ax=ax)
ax.set_title('Distribution of charges for smokers')

ax=f.add_subplot(122)
sns.distplot(data[(data.smoker == 0)]['charges'],color='b',ax=ax)
ax.set_title('Distribution of charges for non-smokers')

실제 분포도를 보시면 흡연자는 더 큰 비용을 발생하고 있고, 비 흡연자는 더 큰 비중을 차지하고 있음.

흡연 여부, 성별 별로 구분해서 count를 수행

In [ ]:

sns.catplot(x="smoker", kind="count",hue = 'sex', palette="pink", data=data)

앞 부분 encoding에서 여성을 "0"로 하고, 남성을 "1"으로 encoding 하였음.
위 그래프 확인 결과 비 흡연자가 흡연자 대비 더 많으며, 흡연자 중에서 남성의 비율이 더 큼.

흡연과 성별에 대하여 좀 더 디테일하게 바이올린 그래프를 통하여 q1, q3, median수치를 확인할 수 있음.

In [ ]:
sns.catplot(x="sex", y="charges", hue="smoker",
            kind="violin", data=data, palette = 'magma')

여성만 필터링해서 흡연 여부에 따라서 박스 플롯으로 분포를 확인 가능.

In [ ]:
pl.figure(figsize=(12,5))
pl.title("Box plot for charges of women")
sns.boxplot(y="smoker", x="charges", data =  data[(data.sex == 1)] , orient="h", palette = 'magma')

남성만 필터링해서 흡연 여부에 따라서 박스 플롯으로 분포를 확인 가능.

In [ ]:
pl.figure(figsize=(12,5))
pl.title("Box plot for charges of men")
sns.boxplot(y="smoker", x="charges", data =  data[(data.sex == 0)] , orient="h", palette = 'rainbow')

다음, 나이 별로 charge에 어떤 영향을 주고, 분포가 어떻게 되는지를 확인 함.

In [ ]:
pl.figure(figsize=(12,5))
pl.title("Distribution of age")
ax = sns.distplot(data["age"], color = 'g')

확인 결과 데이터중 20세 미만의 환자가 있고.. 18세는 가장 어린 나이이고, 64세는 가장 큰 나이 임.
추가로 18세 환자중 흡연자가 있는지를 확인 함.

In [ ]:
sns.catplot(x="smoker", kind="count",hue = 'sex', palette="rainbow", data=data[(data.age == 18)])
pl.title("The number of smokers and non-smokers (18 years old)")

놀랍게도 18세에 흡연자가 있음.
그러면 18세 환자중 흡연 여부가 charge에 큰 영향을 주는지를 분석 함.

In [ ]:
pl.figure(figsize=(12,5))
pl.title("Box plot for charges 18 years old smokers")
sns.boxplot(y="smoker", x="charges", data = data[(data.age == 18)] , orient="h", palette = 'pink')

위 박스 플롯에 따르면 18세 환자중 흡연자들은 charge가 엄첨 크게 발생되고 있음.

비 흡연자를 나이별 로 charge를 산포도로 찍어 줌.

In [ ]:
g = sns.jointplot(x="age", y="charges", data = data[(data.smoker == 0)],kind="kde", color="m")
g.plot_joint(pl.scatter, c="w", s=30, linewidth=1, marker="+")
g.ax_joint.collections[0].set_alpha(0)
g.set_axis_labels("$X$", "$Y$")
ax.set_title('Distribution of charges and age for non-smokers')

흡연자를 나이별 로 charge를 산포도로 찍어 줌.

In [ ]:
g = sns.jointplot(x="age", y="charges", data = data[(data.smoker == 1)],kind="kde", color="c")
g.plot_joint(pl.scatter, c="w", s=30, linewidth=1, marker="+")
g.ax_joint.collections[0].set_alpha(0)
g.set_axis_labels("$X$", "$Y$")
ax.set_title('Distribution of charges and age for smokers')

비 흡연자 산포도는 아래와 같이 다른 형태로 구현 가능

In [ ]:
#non - smokers
p = figure(plot_width=500, plot_height=450)
p.circle(x=data[(data.smoker == 0)].age,y=data[(data.smoker == 0)].charges, size=7, line_color="navy", fill_color="pink", fill_alpha=0.9)

show(p)

흡연자 산포도는 아래와 같이 다른 형태로 구현 가능

In [ ]:
#smokers
p = figure(plot_width=500, plot_height=450)
p.circle(x=data[(data.smoker == 1)].age,y=data[(data.smoker == 1)].charges, size=7, line_color="navy", fill_color="red", fill_alpha=0.9)
show(p)

흡연자 비흡연자를 나이별로 한개 그래프에 그려줌.

In [ ]:

sns.lmplot(x="age", y="charges", hue="smoker", data=data, palette = 'inferno_r', size = 7)
ax.set_title('Smokers and non-smokers')

위 그래프를 확인한 결과 비 흡연자는 나이가 증가 되면서 치료비가 증가되는 추세 임.

하지만 흡연자 치료 비용은 나이와 큰 관계가 없는 것으로 보임

혹시 bmi가 치료 비용과 관계가 있는지를 분석해야함.

bmi의 분포를 그려 줌.

In [ ]:
pl.figure(figsize=(12,5))
pl.title("Distribution of bmi")
ax = sns.distplot(data["bmi"], color = 'm')

환자의 평균 bmi는 30이고, 아래와 같이 구글을 통하여 건강한 bmi가 얼마인지를 확인함.
![![image.png](attachment:image.png)](http://1j4g1pasf991x0osxuqz6d10.wpengine.netdna-cdn.com/wp-content/uploads/2017/03/BMI-CHART-1024x791.png)
30이 초과 되면 비만이라고 볼 수 있음.
bmi30이상과 30이하의 분포를 확인함.

bmi 30이상 분포

In [ ]:
pl.figure(figsize=(12,5))
pl.title("Distribution of charges for patients with BMI greater than 30")
ax = sns.distplot(data[(data.bmi >= 30)]['charges'], color = 'm')

bmi 30이하 분포

In [ ]:
pl.figure(figsize=(12,5))
pl.title("Distribution of charges for patients with BMI less than 30")
ax = sns.distplot(data[(data.bmi < 30)]['charges'], color = 'b')

위 비교 결과 bmi가 30이 초과 될 경우 훨씬 많은 비용이 발생함.

bmi에 대한 charge분포를 산포도로 그려줌.

In [ ]:
g = sns.jointplot(x="bmi", y="charges", data = data,kind="kde", color="r")
g.plot_joint(pl.scatter, c="w", s=30, linewidth=1, marker="+")
g.ax_joint.collections[0].set_alpha(0)
g.set_axis_labels("$X$", "$Y$")
ax.set_title('Distribution of bmi and charges')


흡연 여부에 따라 다른 형태로 산포도를 그려 줌.

In [ ]:
pl.figure(figsize=(10,6))
ax = sns.scatterplot(x='bmi',y='charges',data=data,palette='magma',hue='smoker')
ax.set_title('Scatter plot of charges and bmi')

sns.lmplot(x="bmi", y="charges", hue="smoker", data=data, palette = 'magma', size = 8)

다음, 환자들이 애를 몇명 갖고 있는지를 분석함.

In [ ]:
sns.catplot(x="children", kind="count", palette="ch:.25", data=data, size = 6)

대부분 환자들은 애가 없으며, 일부눈 애를 5명 갖고 있음.

혹시 애가 있는 환자가 흡연 하는지를 확인 함.

In [ ]:
sns.catplot(x="smoker", kind="count", palette="rainbow",hue = "sex",
            data=data[(data.children > 0)], size = 6)
ax.set_title('Smokers and non-smokers who have childrens')

확인 결과 애가 있어도 흡연 하는 환자가 있음, 하지만 비 흡연자가 더 많음.

이제 부터 본격적으로 예측 분석을 진행함.

linear regression 모델로 수행.


sklearn에서 패키지를 임포트 함.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor

모델을 트레이닝함.

In [ ]:
x = data.drop(['charges'], axis = 1)
y = data.charges

x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = 0)
lr = LinearRegression().fit(x_train,y_train)

y_train_pred = lr.predict(x_train)
y_test_pred = lr.predict(x_test)

print(lr.score(x_test,y_test))

normalization을 하지 않고 모든 칼람을 feature로 돌린 결과도 정확도가 나쁘지 않음.

사실 일반적으로 데이터 전처리 작업을 잘 해야만 모델 결과가 잘 나옴.

그러면 linear regression보다 한 단계 더 나아가서 다항식으로 모델을 트레이닝 함.

In [ ]:
X = data.drop(['charges','region'], axis = 1)
Y = data.charges



quad = PolynomialFeatures (degree = 2)
x_quad = quad.fit_transform(X)

X_train,X_test,Y_train,Y_test = train_test_split(x_quad,Y, random_state = 0)

plr = LinearRegression().fit(X_train,Y_train)

Y_train_pred = plr.predict(X_train)
Y_test_pred = plr.predict(X_test)

print(plr.score(X_test,Y_test))

모델 결과가 기존 보다 훨씬 좋게 나옴.

다음 모델 평가 진행.

In [ ]:
forest = RandomForestRegressor(n_estimators = 100,
                              criterion = 'mse',
                              random_state = 1,
                              n_jobs = -1)
forest.fit(x_train,y_train)
forest_train_pred = forest.predict(x_train)
forest_test_pred = forest.predict(x_test)

print('MSE train data: %.3f, MSE test data: %.3f' % (
mean_squared_error(y_train,forest_train_pred),
mean_squared_error(y_test,forest_test_pred)))
print('R2 train data: %.3f, R2 test data: %.3f' % (
r2_score(y_train,forest_train_pred),
r2_score(y_test,forest_test_pred)))

예측 결과에 대하여 산포도로 확인 함.

In [ ]:
pl.figure(figsize=(10,6))

pl.scatter(forest_train_pred,forest_train_pred - y_train,
          c = 'black', marker = 'o', s = 35, alpha = 0.5,
          label = 'Train data')
pl.scatter(forest_test_pred,forest_test_pred - y_test,
          c = 'c', marker = 'o', s = 35, alpha = 0.7,
          label = 'Test data')
pl.xlabel('Predicted values')
pl.ylabel('Tailings')
pl.legend(loc = 'upper left')
pl.hlines(y = 0, xmin = 0, xmax = 60000, lw = 2, color = 'red')
pl.show()